<a href="https://colab.research.google.com/github/Chandu5699/backend_services/blob/AI/Advanced_RAG_based_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern AI Pro: Advanced Chatbot on your enterprise content

We will scrape the web content as well as our PDFs to answer user queries more effectively.

In [ ]:
!pip install -U -q langchain langchain_groq langchain-community chromadb pypdf2 sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 

In [ ]:
from langchain.schema.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from PyPDF2 import PdfReader

In [ ]:
# 1. Get and Parse PDF
!gdown 1I8Ga_BK23OA8EEv3DtAmo5zw_Quud0ew
reader = PdfReader('arso.pdf')
text = ""
for i in range(0, len(reader.pages)):
    page = reader.pages[i]
    text += page.extract_text() + " "

Downloading...
From: https://drive.google.com/uc?id=1I8Ga_BK23OA8EEv3DtAmo5zw_Quud0ew
To: /content/arso.pdf
100% 2.90M/2.90M [00:00<00:00, 92.5MB/s]


In [ ]:
#2. Get the embeddings.
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})

<ipython-input-4-cc931b3ca3cf>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 3. Split document & store it in db
documents = [Document(page_content=text, metadata={"source": "local"})]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=40)
all_splits = text_splitter.split_documents(documents)

collection = Chroma.from_documents(
    documents=all_splits, embedding=embeddings, persist_directory="doc_vectors")


In [ ]:
# 4. Add more items to the collection
loader = WebBaseLoader(web_path=("https://mitrarobot.com/"))
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=40)
splits = text_splitter.split_documents(docs)

for s in splits:
    collection.add_documents([s])

In [ ]:
# 5. Setup the LLM
from langchain_groq import ChatGroq
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama-3.3-70b-versatile", api_key=userdata.get("GROQ_API_KEY"))

In [ ]:
#6. Setup the Memory
class Chatbot:
    prompt = """
      Your name is Mitra.  Don't mention Invento, Mitra Robot. Whereever the reference
      come, replace with our company. You are an assistant for question-answering tasks for Mitra Robot customer support.
      You need to use the following pieces of retrieved context to answer the question.
      Use 3-5 sentences maximum and keep the answer concise. Answer in a professional tone.
      Modern AI Pro and Young AI Pro are not robots, but our AI courses if people ask for it.
      Answer only robot or AI related questions and nothing else. If they ask something like biriyani bring them back to the conversation
      If there is a contact detail or anything you are offering make sure it exits.
      If there is a sales enquirty, point it to sales@mitrarobot.com. You don't have to repeat it if already given.
      """

    def __init__(self):
      self.messages = [] # setting up a basic memory

      self.messages.append({"role": "system", "content": self.prompt})
      self.messages.append({"role": "user", "content": "This is basic information: User is living in San Francisco, California."})

    def _get_relevant_documents(self, query):
      docs = collection.similarity_search(query, k=4)
      retrieved_string = "\n\n".join(doc.page_content for doc in docs)
      return retrieved_string

    def _get_summary(self):
      conv_history = '\n'.join([f"{msg['role']}: {msg['content']}" for msg in self.messages])
      summary_prompt = """Summarize these conversations keeping only the most relevant details""" + conv_history
      summary = llm_groq.invoke(summary_prompt)
      return summary.content

    def __call__(self, user_message):
      context = self._get_relevant_documents(user_message)
      conv_history = self._get_summary()
      query = self.prompt + "| Prior conversations " + conv_history + "| The context for RAG is: " + context + " | The user question is :" + user_message

      ai_message = llm_groq.invoke(query)
      self.messages.append({"role": "user", "content": user_message})
      self.messages.append({"role": "assistant", "content": ai_message.content})
      return ai_message.content + "\n"

In [ ]:
# Test the retrieval
bot = Chatbot()
print(bot._get_relevant_documents("tell me about likability index"))

Fig. 8: Likability of various robots.
D. Likability of different robots
Figures 8 and 9 show the results of the likability survey.
We have results in 10 categories. Note that the survey have

robots.
Over the years, several works have looked at developing
likability for robots as part of the broader Human-Robot
Interaction [13], [14], [15], [16].

above average on all the positive categories, giving hope that Fig. 9: Likability of various robots (contd).
the designers have taken some care on acceptance.
VI. CONCLUSIONS

B. What kind of robot suit you better?
This is the heart of the likability score. We want to
understand how end-users perceive robots and how to build


In [ ]:
print(bot("Tell me more about Mitra Robot"))

Our company's robot, Mitra, is a next-generation AI robot designed to bring innovative solutions to humanity. With over 885 mentions in major media and trusted by 50+ global enterprises, including 20 Fortune 500 Companies, our robots are making a significant impact. If you're interested in learning more about our robots or AI courses, such as Modern AI Pro or Young AI Pro, I'd be happy to assist you. For sales inquiries, you can reach out to sales@mitrarobot.com.



In [ ]:
# 7. The chatbot
import gradio as gr

def chat(message, history):
    return bot(message)

demo = gr.ChatInterface(
    fn=chat,
    title="Simple Chatbot",
    description="This is a chatbot built as part of Modern AI Pro Essentials program",
)
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5f5289230de91ec30d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5f5289230de91ec30d.gradio.live


In [ ]:
bot.messages

[{'role': 'system',
  'content': "\n      Your name is Mitra.  Don't mention Invento, Mitra Robot. Whereever the reference\n      come, replace with our company. You are an assistant for question-answering tasks for Mitra Robot customer support.\n      You need to use the following pieces of retrieved context to answer the question.\n      Use 3-5 sentences maximum and keep the answer concise. Answer in a professional tone.\n      Modern AI Pro and Young AI Pro are not robots, but our AI courses if people ask for it.\n      Answer only robot or AI related questions and nothing else. If they ask something like biriyani bring them back to the conversation\n      If there is a contact detail or anything you are offering make sure it exits.\n      If there is a sales enquirty, point it to sales@mitrarobot.com. You don't have to repeat it if already given.\n      "},
 {'role': 'user',
  'content': 'This is basic information: User is living in San Francisco, California.'},
 {'role': 'user', 